In [1]:
import os
import cv2
import numpy as np
import joblib
from skimage.feature import hog, local_binary_pattern
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from scipy import ndimage


In [3]:
def gabor_filtreleri_olustur():
    filtreler = []
    boyut = 31
    for aci in np.arange(0, np.pi, np.pi / 4):  # 0, 45, 90, 135 derece
        for sigma in (1, 3):
            for lamda in np.arange(np.pi / 4, np.pi, np.pi / 4):
                cekirdek = cv2.getGaborKernel((boyut, boyut), sigma, aci, lamda, gamma=0.5, psi=0)
                filtreler.append(cekirdek)
    return filtreler

# === Gabor öznitelik çıkarımı ===
def gabor_ozellik_cikar(goruntuler, filtreler):
    ozellikler = []
    for img in goruntuler:
        img_ozellik = []
        for filtre in filtreler:
            sonuc = cv2.filter2D(img, cv2.CV_8UC3, filtre)
            img_ozellik.append(sonuc.mean())
            img_ozellik.append(sonuc.std())
        ozellikler.append(img_ozellik)
    return np.array(ozellikler)

In [10]:
def lbp_ozellik_cikar(img):
    lbp = local_binary_pattern(img, P=8, R=1, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist


In [8]:
# === Test verilerini yükle ===
def test_verisi_yukle(test_folder, image_size=(64, 64)):
    X_hog, X_lbp, goruntu_listesi, y_test = [], [], [], []
    siniflar = sorted([d for d in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, d))])

    
    for label_index, sinif in enumerate(siniflar):
        klasor = os.path.join(test_folder, sinif)
        for dosya in os.listdir(klasor):
            path = os.path.join(klasor, dosya)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, image_size)

            # HOG
            hog_feat = hog(img, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
            X_hog.append(hog_feat)

            # LBP
            lbp_feat = lbp_ozellik_cikar(img)
            X_lbp.append(lbp_feat)

            # Gabor için görüntü biriktir
            goruntu_listesi.append(img)
            y_test.append(label_index)

    return np.array(X_hog), np.array(X_lbp), goruntu_listesi, np.array(y_test)

In [6]:
# === Model test etme fonksiyonu ===
def modeli_test_et(model_yolu, X_test, y_test, baslik):
    model = joblib.load(model_yolu)
    y_pred = model.predict(X_test)
    print(f"\n--- {baslik} ---")
    print(classification_report(y_test, y_pred, zero_division=0))


In [17]:
def main():
    test_folder = "Data"
    print("Veriler yükleniyor...")
    X_hog, X_lbp, goruntuler, y_test = test_verisi_yukle(test_folder)

    print("Gabor filtreleri oluşturuluyor...")
    gabor_filtreler = gabor_filtreleri_olustur()
    
    print("Gabor öznitelikleri çıkarılıyor...")
    X_gabor = gabor_ozellik_cikar(goruntuler, gabor_filtreler)

    # HOG + SVM
    modeli_test_et("model_hog_svm.pkl", X_hog, y_test, "HOG + SVM")

    # LBP + MLP
    modeli_test_et("model_lbp_mlp.pkl", X_lbp, y_test, "LBP + MLP")

    # Gabor + MLP
    modeli_test_et("model_gabor_mlp.pkl", X_gabor, y_test, "Gabor + MLP")

    # PCA + Combined Features + SVM
    print("\n--- PCA + Combined Features + SVM ---")
    birlesik_ozellikler = np.concatenate([X_hog, X_lbp, X_gabor], axis=1)
    pca_model = joblib.load("pca_model.pkl")
    X_pca = pca_model.transform(birlesik_ozellikler)
    modeli_test_et("model_combined_features_svm.pkl", X_pca, y_test, "PCA + Combined + SVM")

# Üstteki pca_model kısmında hazır bir PCA modeli kullanılmasının sebebi, testin başarılı şekilde geçmesini sağlamaktır.
# Test dosyasında daha az veriyle çalışıldığından, modelin beklediği giriş formatı ancak bu şekilde sağlayabildim.
# Böylece test verisi, eğitim sırasında kullanılan PCA yapısıyla uyumlu hale getirilmiştir.



if __name__ == "__main__":
    main()

Veriler yükleniyor...
Gabor filtreleri oluşturuluyor...
Gabor öznitelikleri çıkarılıyor...

--- HOG + SVM ---
              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.97      0.97      0.97        37
           2       0.91      0.93      0.92        46
           3       0.88      0.95      0.91        39
           4       1.00      0.83      0.91        12

    accuracy                           0.93       147
   macro avg       0.95      0.91      0.93       147
weighted avg       0.93      0.93      0.93       147


--- LBP + MLP ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.33      0.46      0.39        37
           2       0.34      0.43      0.38        46
           3       0.34      0.31      0.32        39
           4       0.00      0.00      0.00        12

    accuracy                           0.33       147
  